In [193]:
import os
import csv
import json
import pandas as pd
from tqdm import tqdm
from pprint import pprint

DATA_PATH = "./data"

In [186]:
def create_data_frame(match_data):
    gameId = []
    gameDuration = []
    championsPicked = []
    goldDifference = []
    damageDifference = []
    win = []
    
    for version in match_data.keys():
        for data in match_data[version]:
            gameId.append(data["metadata"]["matchId"])

            if version[:5] >= "11.20":
                gameDuration.append(data["info"]["gameDuration"])
            else:
                gameDuration.append(data["info"]["gameDuration"] // 1000)

            championName = []
            goldEarned = []
            totalDamageDealtToChampions = []

            for participants in data["info"]["participants"]:
                championName.append(participants["championName"])
                goldEarned.append(participants["goldEarned"])
                totalDamageDealtToChampions.append(
                    participants["totalDamageDealtToChampions"]
                )

            championsPicked.append(championName)
            goldDifference.append(sum(goldEarned[:5]) - sum(goldEarned[5:]))
            damageDifference.append(
                sum(totalDamageDealtToChampions[:5])
                - sum(totalDamageDealtToChampions[5:])
            )
            win.append(data["info"]["participants"][0]["win"])

    df = pd.DataFrame(
        {
            "game_Id": gameId,
            "game_duration": gameDuration,
            "champions_picked": championsPicked,
            "gold_difference": goldDifference,
            "damage_difference": damageDifference,
            "win": win,
        }
    )
    return df

In [189]:
with open(os.path.join(DATA_PATH, "MASTER_I_MatchData_1_500.json")) as json_data:
    match_data = json.load(json_data)
    df = create_data_frame(match_data)

df

,game_Id,game_duration,champions_picked,gold_difference,damage_difference,win
0,KR_5596108571,918,"[Shen, JarvanIV, Galio, Lucian, Lulu, Sion, Gr...",-15531,-11073,False
1,KR_5595513300,1570,"[Shen, Talon, Viktor, Jinx, Alistar, Jayce, Le...",-13031,-15094,False
2,KR_5595151915,1338,"[Shen, MasterYi, Vex, Xayah, Leona, Cassiopeia...",11693,13474,True
3,KR_5594992629,1406,"[Karthus, Talon, Shen, Xayah, Leona, Fiora, Gr...",-10947,-6186,False
4,KR_5591864726,1390,"[Jayce, Talon, Zoe, Caitlyn, Alistar, Shen, Le...",8354,25967,True
...,...,...,...,...,...,...
36346,KR_5339312295,2151,"[Irelia, JarvanIV, Leblanc, Ashe, Thresh, Jayc...",6406,3537,True
36347,KR_5339206901,1230,"[Sylas, XinZhao, Yone, Ezreal, Karma, Riven, F...",-5069,-6023,False
36348,KR_5339100477,1681,"[Teemo, LeeSin, Leblanc, Kaisa, Pyke, Jax, Vie...",-11529,-16216,False
36349,KR_5339026932,1170,"[Aphelios, Viego, Sylas, Camille, Leona, Volib...",2872,11625,True


In [196]:
all_tier_df = []
for fileName in tqdm(os.listdir(DATA_PATH)):
    if "json" in fileName:
        with open(os.path.join(DATA_PATH, fileName)) as json_data:
            match_data = json.load(json_data)
            df = create_data_frame(match_data)
            all_tier_df.append(df)

100%|██████████| 14/14 [05:23<00:00, 23.12s/it]


In [198]:
df_concat = pd.concat(all_tier_df, ignore_index=True)
df_concat

,game_Id,game_duration,champions_picked,gold_difference,damage_difference,win
0,KR_5593619149,1067,"[Fiora, Zed, Akali, Ezreal, Maokai, Jayce, Tal...",18238,25729,True
1,KR_5592304230,1325,"[Kennen, Karthus, Akshan, Ezreal, Leona, Camil...",-6523,-2185,False
2,KR_5592027955,1440,"[Mordekaiser, LeeSin, Viktor, Caitlyn, Pyke, C...",17032,22317,True
3,KR_5591815606,1439,"[Yasuo, Qiyana, Vex, Alistar, Gangplank, Sylas...",14882,21889,True
4,KR_5591266319,1121,"[Jax, Talon, Katarina, Jinx, Lulu, Malphite, W...",-4522,-13935,False
...,...,...,...,...,...,...
339956,KR_5572036428,1034,"[Irelia, Graves, Leblanc, Aphelios, Lulu, Jayc...",14337,12984,True
339957,KR_5571939433,1595,"[Yone, Viego, Syndra, Jhin, Zyra, Graves, Nida...",-9060,-2191,False
339958,KR_5571935533,1597,"[Irelia, Viego, Vex, Lucian, Bard, Yasuo, Dian...",9416,5245,True
339959,KR_5571932202,937,"[Camille, Lillia, Yone, Draven, Thresh, Renekt...",-14884,-18677,False


In [202]:
df_concat = df_concat.drop_duplicates(subset=["game_Id"])
df_concat

,game_Id,game_duration,champions_picked,gold_difference,damage_difference,win
0,KR_5593619149,1067,"[Fiora, Zed, Akali, Ezreal, Maokai, Jayce, Tal...",18238,25729,True
1,KR_5592304230,1325,"[Kennen, Karthus, Akshan, Ezreal, Leona, Camil...",-6523,-2185,False
2,KR_5592027955,1440,"[Mordekaiser, LeeSin, Viktor, Caitlyn, Pyke, C...",17032,22317,True
3,KR_5591815606,1439,"[Yasuo, Qiyana, Vex, Alistar, Gangplank, Sylas...",14882,21889,True
4,KR_5591266319,1121,"[Jax, Talon, Katarina, Jinx, Lulu, Malphite, W...",-4522,-13935,False
...,...,...,...,...,...,...
339948,KR_5572754029,1757,"[Renekton, LeeSin, Taliyah, Aphelios, Thresh, ...",-9120,-29188,False
339954,KR_5572263228,936,"[Gragas, Ryze, Talon, Xayah, Taliyah, Irelia, ...",10057,15017,True
339955,KR_5572059501,1158,"[Renekton, Nidalee, Ryze, Jinx, Thresh, Gragas...",13192,4258,True
339958,KR_5571935533,1597,"[Irelia, Viego, Vex, Lucian, Bard, Yasuo, Dian...",9416,5245,True


In [204]:
df_concat.to_csv("Champions_picked.csv", index=False)